### Have a model to import in html to make the users write a sentence and make the model react

In [ ]:
from transformers import GPT2Model
from transformers import GPT2Tokenizer
import torch
import torch.nn as nn

gpt_model = GPT2Model.from_pretrained('gpt2')
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
tokenizer.pad_token = tokenizer.eos_token  # GPT2 does not have a pad token by default

def tokenize(batch):
    return tokenizer(batch['Summaries'], padding=True, truncation=True, max_length=128)


class GPTClassifier(nn.Module):
    def __init__(self, gpt_model, num_classes):
        super(GPTClassifier, self).__init__()
        self.gpt = gpt_model
        self.dropout = nn.Dropout(0.1)
        self.classifier = nn.Linear(self.gpt.config.hidden_size, num_classes)
    
    def forward(self, input_ids, attention_mask):
        outputs = self.gpt(input_ids=input_ids, attention_mask=attention_mask)
        # Using the last hidden state of the last token
        last_hidden_state = outputs.last_hidden_state
        pooled_output = last_hidden_state[:, -1, :]  # (batch_size, hidden_size)
        pooled_output = self.dropout(pooled_output)
        logits = self.classifier(pooled_output)
        return logits

num_classes = 2  # 'spam' or 'ham'
model = GPTClassifier(gpt_model, num_classes)
model.load_state_dict(torch.load('/Users/shrinidhivelan/Desktop/ADA/ada-2024-project-theadavengers/best_model_state.bin'))
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

def predict_text(text):
    model.eval()
    inputs = tokenizer(text, return_tensors='pt', truncation=True, max_length=128, padding=True)
    input_ids = inputs['input_ids'].to(device)
    attention_mask = inputs['attention_mask'].to(device)
    
    with torch.no_grad():
        outputs = model(input_ids, attention_mask)
        _, preds = torch.max(outputs, dim=1)
        label = 1 if preds.item() == 1 else 0
    return label

# Trial
sample_text = 'Conflict and Conquest: Central to a feminine plot is a clear conflict, often framed as a battle or competition. The hero may confront external forces (like friends, societal expectations, or natural obstacles) or internal strength (like personal flaws or emotional growth).'
print(f"Message: {sample_text}")
print(f"Prediction: {predict_text(sample_text)}")

/var/folders/p6/m1p5k4zj0yn9mk66d_dh7pn40000gn/T/ipykernel_22322/1751070928.py:34: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load('/Users/shr

Message: Conflict and Conquest: Central to a feminine plot is a clear conflict, often framed as a battle or competition. The hero may confront external forces (like friends, societal expectations, or natural obstacles) or internal strength (like personal flaws or emotional growth).
Prediction: 0
